<a href="https://colab.research.google.com/github/Diooonis2Syracuse/ML/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
%pip install torch
import torch
import torch.nn as nn
import numpy as np

In [ ]:
import torchvision
from torchvision import transforms
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
    ])

mnist_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True,
                                           transform=transform,
                                           download=True
                                           )

from torch.utils.data import Subset

mnist_valid_dataset = Subset(mnist_dataset,
                             torch.arange(10000))

mnist_train_dataset = Subset(mnist_dataset,
                             torch.arange(10000, len(mnist_dataset))
                             )

mnist_test_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=False,
                                           transform=transform,
                                           download=False
                                           )

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 150390343.51it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 48590330.45it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 66347122.90it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 9611770.32it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)

train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset,
                      batch_size,
                      shuffle=True)

In [ ]:
model = nn.Sequential()
model.add_module('conv1',
                 nn.Conv2d(
                     in_channels=1, out_channels=32,
                     kernel_size=5, padding=2)
                 )
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module('conv2',
                 nn.Conv2d(in_channels=32, out_channels=64,
                           kernel_size=5, padding=2)
                 )
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

model.add_module('flatten', nn.Flatten())

model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10))

In [ ]:
model

Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (relu3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimazer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, num_epochs, train_dl, valid_dl):
  loss_hist_train = [0] * num_epochs
  accuracy_hist_train = [0] * num_epochs
  loss_hist_valid = [0] * num_epochs
  accuracy_hist_valid = [0] * num_epochs

  for epoch in range(num_epochs):
    model.train()
    for x_batch, y_batch in train_dl:
      pred = model(x_batch)
      loss = loss_fn(pred, y_batch)
      loss.backward()
      optimazer.step()
      optimazer.zero_grad()
      loss_hist_train[epoch] = loss_hist_train[epoch] + loss.item() * y_batch.size(0)
      is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
      accuracy_hist_train[epoch] = accuracy_hist_train[epoch] + is_correct.sum()

    loss_hist_train[epoch] = loss_hist_train[epoch]/len(train_dl.dataset)
    accuracy_hist_train[epoch] = accuracy_hist_train[epoch]/len(train_dl.dataset)

    model.eval()
    with torch.no_grad():
      for x_batch, y_batch in valid_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss_hist_valid[epoch] = loss_hist_valid[epoch] + loss.item() * y_batch.size(0)
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_valid[epoch] = accuracy_hist_valid[epoch] + is_correct.sum()

    loss_hist_valid[epoch] = loss_hist_valid[epoch] / len(valid_dl.dataset)
    accuracy_hist_valid[epoch] = accuracy_hist_valid[epoch] / len(valid_dl.dataset)

    print(f'Точность эпохи {epoch +1}:'
          f'{accuracy_hist_train[epoch]:.4f} val_accuracy:'
          f'{accuracy_hist_valid[epoch]:.4f}')
  return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid

In [ ]:
torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, train_dl, valid_dl)

Точность эпохи 1:0.9725 val_accuracy:0.9814
Точность эпохи 2:0.9848 val_accuracy:0.9877
Точность эпохи 3:0.9892 val_accuracy:0.9887
Точность эпохи 4:0.9917 val_accuracy:0.9874
Точность эпохи 5:0.9932 val_accuracy:0.9891
Точность эпохи 6:0.9949 val_accuracy:0.9902
Точность эпохи 7:0.9949 val_accuracy:0.9907
Точность эпохи 8:0.9961 val_accuracy:0.9901
Точность эпохи 9:0.9960 val_accuracy:0.9909
Точность эпохи 10:0.9967 val_accuracy:0.9909
Точность эпохи 11:0.9968 val_accuracy:0.9892
Точность эпохи 12:0.9974 val_accuracy:0.9911
Точность эпохи 13:0.9978 val_accuracy:0.9914
Точность эпохи 14:0.9976 val_accuracy:0.9897
Точность эпохи 15:0.9970 val_accuracy:0.9894
Точность эпохи 16:0.9980 val_accuracy:0.9903
Точность эпохи 17:0.9985 val_accuracy:0.9907
Точность эпохи 18:0.9981 val_accuracy:0.9887
Точность эпохи 19:0.9982 val_accuracy:0.9901
Точность эпохи 20:0.9984 val_accuracy:0.9905


([0.0882371062618494,
  0.04730720713421702,
  0.03411389607708901,
  0.025528604662325233,
  0.021716197602236642,
  0.0165023057138687,
  0.015643613930549426,
  0.011610090701035497,
  0.01255876785994129,
  0.01032725062551588,
  0.009584582094332,
  0.008156965940237206,
  0.006915506733695184,
  0.007321487053096721,
  0.008927498331076407,
  0.005891857123817153,
  0.005576912495697361,
  0.005895226613018485,
  0.007311772083033184,
  0.005135086527624572],
 [0.05909077031351626,
  0.03923694217652082,
  0.039730981132667514,
  0.04110037376619875,
  0.040291618224140256,
  0.04019515080556739,
  0.040218639341494056,
  0.03929333222032583,
  0.03661107099449728,
  0.04374711890520593,
  0.04562113742052461,
  0.041390672050845026,
  0.05120430311461778,
  0.060920678308043805,
  0.05582455277484405,
  0.06184263156649081,
  0.06188923011611099,
  0.06820789397303349,
  0.06805102658168669,
  0.06285693832663944],
 [tensor(0.9725),
  tensor(0.9848),
  tensor(0.9892),
  tensor(0